# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, deque
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import cPickle as pickle
import slate 
import re
from fenaroli_parser.parser import text_from_pdf, make_ingredient_compound_dict

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [6]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [7]:
cleaned_joined = joined.replace('.', ' ')

## RE for flavor compounds and ingredients

## Here I have the material I need to build the ingredient:compound mapping

In [175]:
def flav_ingr_mapper(raw_text):
    
    flavor_dict = defaultdict(list)
    ignore_terms = ['EINECS No :', 'EINECS No ']
    re.MULTILINE
    ingr_pattern = re.compile('Natural occurrence:(.+)')
    comp_pattern = re.compile('(.*[A-Z]{6,20}.*)')
    
    blobs = raw_text.split('\n\n')
    comp_q = deque([], maxlen=4)
    comp_list = []
    ing = None
    comp = None
    for i in range(len(blobs)):
        text = blobs[i].replace('\n', '')
        potential_ing = re.findall(ingr_pattern, text)
        potentiacomp = re.findall(comp_pattern, text)
        if comp and comp[0] not in ignore_terms:
            try:
                if comp_q[-1] != comp[0]:
                    comp_q.append(comp[0])
                    comp_list.append(comp[0])
            except IndexError:
                comp_q.append(comp[0])
                comp_list.append(comp[0])

        if ing:
            try:
                flavor_dict[comp_q.pop().strip()] = ing[0].strip()
            except IndexError:
                pass
            
    return comp_list, flavor_dict

In [231]:
re.MULTILINE
ingredient_pattern = re.compile('Natural\s+occurrence:(.+)')

blobs = cleaned_joined.split('\n\n')
comp_list = []
for i in range(int(len(blobs)*0.2), len(blobs)):
    text = blobs[i].replace('\n', '')
    potential_ing = re.findall(ingredient_pattern, text)
    if potential_ing:
        comp_list.append(potential_ing[0].strip())

In [22]:
flav_comp_dict = text_from_pdf(cleaned_joined)

In [9]:
# print len(set(comp_list))
# print len(comp_list)
print len(flav_comp_dict.keys())

982


In [11]:
flav_comp_dict['ISOBUTYL 2-BUTENOATE']

flav_comp_dict['ISOBUTYL BUTYRATE']

flav_comp_dict['ISOBUTYL CINNAMATE']

flav_comp_dict['TRIACETIN']

'Reported found in kumquat peel oil'

This is the compound - ingredient mapping I've been striving for

In [320]:
with open('./flavor_compund_map.pkl', 'w') as f:
    pickle.dump(flav_comp_dict, f)

In [76]:
ing_comp_dict = make_ingredient_compound_dict(flav_comp_dict)

In [82]:
for key, val in ing_comp_dict.iteritems():
    print key, val, '\n'

EUGENYL METHYL ETHER ['reported', 'the essential oils of myrtaceae', 'luraceae', 'it was identified originally', 'the essential oil from roots of asarum europaeum l', 'and asarum canadense l', 'subsequently', 'it was identified as the ma', 'constituent of the oil from wood of dacrydium franklinii hook (97 5%)', 'melaleuca bracteata f v m', '(leaves', '90 to 95%)', 'cinnamomum oliveri bail', '(leaves', '90 to 95%)', 'as a minor constituent', 'betel', 'citronella', 'japanese calamus', 'pimenta', 'hyacinth', 'rose', 'basil', 'bay', 'cajeput', 'others', 'heated blackberry', 'pepper', 'lovage seed', 'chervil', 'lemon balm', 'alpinia species', 'clove buds', 'nutmeg', 'pepper', 'mace', 'tarragon', 'ocimum sanctum', 'laurel', 'myrtle leaf', 'berry', 'rosemary', 'pimento berry', 'mastic gum leaf oil'] 

α-DAMASCONE ['black tea'] 

(+/–)-(2,6,6-TRIMETHYL-2-HYDROXYCYCLOHEXYLIDENE) ACETIC ACID γ-LACTONE ['reportedly present', 'lemongrass', 'sweet grass oil'] 

10-UNDECENOIC ACID ['reported found a

In [78]:
ing_comp_dict['beer']

KeyError: 'beer'

In [31]:
ing_comp_dict.keys()

['macadamia nuts',
 'mackerel',
 'acacia flowers; remarkable amounts of eugenol are found in ocimum sanctum (70%)',
 'yellow',
 'in other plants (coffee',
 'asian',
 'grapefruit peel oil',
 'black chokeberry',
 'reported in ceylon citronella oil',
 'apricot  kernel',
 'hyssop',
 'over 300 natural sources including apples',
 'wheat  bread',
 'blackberries',
 'broccoli  (cooked)',
 'oak  musk',
 'in bone oil (together with several methyl homologues); also found in lemon',
 'juice; traces are also',
 'clove leaf',
 'coconut  flavor',
 'reported found as a volatile constituent in coffee',
 'water apple (syzigium spp )',
 'sweet fish',
 'buchu',
 'brandy distillate',
 'r  glabra',
 'raw lean fish',
 'several plants',
 'heater beef fat',
 'thomas  (0 25%)',
 'occurs frequently in nature; in the flowers of spirea ulmaria',
 'guaiene  is  a  bicyclic  sesquiterpene  found  naturally  in  patchouli  oil',
 'figs',
 'clove bud oil',
 'strawberry jam',
 'sheep milk',
 'parmesan  cheese',
 'myrtle

In [ ]:
'Reported found in apple, pear, grape, pineapple, strawberry, raspberry, tomato, black currant, ' \
                     'citrus, onion and potato; also reported found in cocoa leaves, in Mexican goosefoot and in the ' \
                     'oils of coriander and lavender  In trace amounts it has been reportedly identified in the oil ' \
                     'of bitter orange, in distilled wine and in coffee aroma'